In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import blosc2
import time
import psutil
import os
%matplotlib ipympl

# --- Memory profiler ---
def getmem():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

shape1 = (500, 10000)
shape2 = (10, 500, 10000)
a = blosc2.linspace(0, 1, np.prod(shape1), dtype=np.float32, shape=shape1, urlpath="a.b2nd", mode="w")
b = blosc2.linspace(1, 2, np.prod(shape2), dtype=np.float64, shape=shape2, urlpath="b.b2nd", mode="w")

# Now, let's create an expression that involves `a` and `b`, called `c`.
c = a ** 2 + b ** 2 + 2 * a * b + 1
print(c.info)  # at this stage, the expression has not been computed yet

**--------------------Exercise 1------------------**

Blosc2 automatically selects the chunksize and chunk shape when creating arrays to be ajusted to your machine's cache. However, we can modify the chunksizes of ``a`` and ``b`` to be smaller or larger; can you predict how memory usage might change? What about computation time? The latter is probably more difficult to predict.

Extra Credit: Note that we can also specify different compression parameters for the result via the ``cparams`` kwarg of ``compute``.  For example, we can change the codec to `ZLIB`, use the bitshuffle filter, and set the compression level to 9: ``cparams = blosc2.CParams(codec=blosc2.Codec.ZLIB, filters=[blosc2.Filter.BITSHUFFLE], clevel=9)``. What happens to computation time and compression ratio (compare to **i) Computing an expression**) ?


In [ ]:
print(f"Chunks are of size: a - {round(a.chunksize / 1024 ** 2, 2)} MB,  b - {round(b.chunksize / 1024 ** 2, 2)} MB")
# Now check what the chunkshapes for a and b are and regenerate them with different chunks
# Try (100, 5000) for a, (250, 1000) for b

#
## YOUR CODE HERE ##
#

# We have to regenerate expression with new operands
print(
    f"Chunks are now of size: a - {round(a.chunksize / 1024 ** 2, 2)} MB,  b - {round(b.chunksize / 1024 ** 2, 2)} MB")
c = a ** 2 + b ** 2 + 2 * a * b + 1

# Now redo the experiment ii) Writing to disk and see how memory usage and computation time change

#
## YOUR CODE HERE ##
#

# Now reset things back to how they were before
a = blosc2.linspace(0, 1, np.prod(shape1), dtype=np.float32, shape=shape1, urlpath="a.b2nd", mode="w")
b = blosc2.linspace(1, 2, np.prod(shape2), dtype=np.float64, shape=shape2, urlpath="b.b2nd", mode="w")
c = a ** 2 + b ** 2 + 2 * a * b + 1

## Part 2)

In [ ]:
# --- Experiment Setup ---
n_frames = 100  # Raise this for more frames
width, height = np.array((n_frames, n_frames))  # Size of the grid
dtype = np.float64  # Data type for the grid

# --- Coordinate creation ---
x = blosc2.linspace(0, n_frames, n_frames, dtype=dtype)
y = blosc2.linspace(-4 * np.pi, 4 * np.pi, width, dtype=dtype)
z = blosc2.linspace(-4 * np.pi, 4 * np.pi, height, dtype=dtype)
X = blosc2.expand_dims(blosc2.expand_dims(x, 1), 2)  # Shape: (N, 1, 1)
Y = blosc2.expand_dims(blosc2.expand_dims(y, 0), 2)  # Shape: (1, N, 1)
Z = blosc2.expand_dims(blosc2.expand_dims(z, 0), 0)  # Shape: (1, 1, N)

# --- Generate computed data ---
def genexpr(a, b, c):
    time_factor = a * b * 0.001
    R = np.sqrt(b**2 + c**2)
    theta = np.arctan2(c, b)
    return np.sin(R * 3 - time_factor * 2) * np.cos(theta * 3)

def create_video(arr):
    label = 'NumPy' if isinstance(arr, np.ndarray) else 'Blosc2'
    fig = plt.figure(figsize=(8, 8))
    a = arr[:,:,0]
    im = plt.imshow(a, cmap="viridis")
    fig.colorbar(im)
    plt.title("Animated Plot")
    plt.xlabel("X-axis")
    plt.ylabel("Y-axis")
    start_time = time.time()
    def update(frame_num):
        # Evaluate the expression for the current frame on the fly
        a = arr[:, :, frame_num]  # <-------------- perform computation
        im.set_array(a)
        elapsed_time = time.time() - start_time
        plt.title(f"{label}: Frame {frame_num + 1}/{n_frames}, elapsed time = {round(elapsed_time, 2)} s")
        return im,
    ani = FuncAnimation(fig, update, frames=n_frames, interval=10, blit=False, repeat=False)
    plt.show()
    return ani

# --- Run computation and profile time and memory usage ---
def monitor(BLOSC=True):
    m0 = getmem()
    t0 = time.time()
    expr = genexpr(X, Y, Z)[:] if BLOSC else genexpr(X[:], Y[:], Z[:])
    dt = time.time()-t0
    if dt > 1e-3:
        print(f'Operation took {round(dt, 3)} s')
    else:
        print(f'Operation took {round(dt * 1000_000, 1)} μs')
    print(f'Result occupies {round((getmem()-m0))} MB')
    return expr


**----------------Exercise 2----------------**

a) Run the animation with the result of the NumPy calculation (``genexpr(X[:], Y[:], Z[:])``). Since we precompute all frames, you might expect the animation to run faster. Is this true?
You can go back and regenerate the ``expr`` variable to be a ``LazyExpr`` and rerun the Blosc2 animation cell to double check the comparison of the speeds.

b) Now try to run the animation but pass a computed Blosc2 array as the ``arr`` parameter (rather than either the uncomputed ``LazyExpr`` object or a NumPy array). What happens now?

c) What would happen to memory usage and computation time if, in the ``monitor`` function, we modified the computation to return a Blosc2 NDArray rather than a NumPy array?

In [ ]:
## YOUR CODE HERE

## Part 3) Lazy Expressions and Persistent Reductions

**EXERCISE**:

Apply ``blosc2.sin`` to the lazy expression ``a + b``. What is the type of the result? Now apply a reduction (e.g. ``blosc2.sum``) to the lazy expression. What is the type of the result?

In [ ]:
lexpr = a + b
# Apply blosc2.sin to lazy expression
#YOUR CODE HERE
# Apply a reduction to lazy expression
#YOUR CODE HERE

As we can see, the result of the reduction expression is not a ``LazyExpr``, but rather a NumPy array (even though no ``__getitem__`` call has been made). This is because reductions in expressions are always executed "eagerly" (i.e. on creation of the lazy expression). Using strings and the ``lazyexpr`` constructor, we can avoid this:


In [ ]:
# Expression that sums arrays
expression = "sum(a + b, axis=1)"
# Define the operands for the expression
operands = {"a": a, "b": b}
# Create a lazy expression
lazy_expression = blosc2.lazyexpr(expression, operands)

**---------------EXERCISE 3----------------**

a) Check the type of ``lazy_expression`` is indeed a ``LazyExpr``. Try computing the full result and profile the computation. Compare it with the time taken for the eagerly executed version which doesn't use a string.

b) Now try computing a slice of ``lazy_expression`` and see if it runs faster than computing the full result. Would the non-string version also be faster? Would the result even be the same?

In [ ]:
#a)
# YOUR CODE HERE

#b)
# YOUR CODE HERE

## Part 4) Lazy Expression Operands and Storage

We can save lazy expression operands on-disk and then update them - computing any lazy expression in which they appear then gives a new result, since the computation is peformed every time:

In [ ]:
a = blosc2.arange(0, 10, urlpath="a.b2nd", mode="w")
lexpr = a + 1
print(f"Lazyexpr with old a: {lexpr[:]}")
a = blosc2.arange(10, 20, urlpath="a.b2nd", mode="w")
print(f"Lazyexpr with new a: {lexpr[:]}")  # This will compute with the new a

**-------Exercise 4----------**

Now try the same but *without* saving the operands to disk. What happens now? You should see that the behaviour is different. Compare the result of ``id()`` on ``a`` and the values of the ``lexpr.operands`` dict - what do you see? Try saving and reopening ``lexpr`` to disk when the operands are saved on disk (and when they're not) - what happens then?

In [ ]:
a = blosc2.arange(0, 10) # operand stored in memory
print(f"Memory address of a: {id(a)}")

#
## YOUR CODE  HERE
#